In [1]:
import math
import numpy

# epsilon for testing whether a number is close to zero
_EPS = numpy.finfo(float).eps * 4.0

# axis sequences for Euler angles
_NEXT_AXIS = [1, 2, 0, 1]

# map axes strings to/from tuples of inner axis, parity, repetition, frame
_AXES2TUPLE = {
    'sxyz': (0, 0, 0, 0), 'sxyx': (0, 0, 1, 0), 'sxzy': (0, 1, 0, 0),
    'sxzx': (0, 1, 1, 0), 'syzx': (1, 0, 0, 0), 'syzy': (1, 0, 1, 0),
    'syxz': (1, 1, 0, 0), 'syxy': (1, 1, 1, 0), 'szxy': (2, 0, 0, 0),
    'szxz': (2, 0, 1, 0), 'szyx': (2, 1, 0, 0), 'szyz': (2, 1, 1, 0),
    'rzyx': (0, 0, 0, 1), 'rxyx': (0, 0, 1, 1), 'ryzx': (0, 1, 0, 1),
    'rxzx': (0, 1, 1, 1), 'rxzy': (1, 0, 0, 1), 'ryzy': (1, 0, 1, 1),
    'rzxy': (1, 1, 0, 1), 'ryxy': (1, 1, 1, 1), 'ryxz': (2, 0, 0, 1),
    'rzxz': (2, 0, 1, 1), 'rxyz': (2, 1, 0, 1), 'rzyz': (2, 1, 1, 1)}

_TUPLE2AXES = dict((v, k) for k, v in _AXES2TUPLE.items())
def quaternion_matrix(quaternion):
    """Return homogeneous rotation matrix from quaternion.

    >>> R = quaternion_matrix([0.06146124, 0, 0, 0.99810947])
    >>> numpy.allclose(R, rotation_matrix(0.123, (1, 0, 0)))
    True

    """
    q = numpy.array(quaternion[:4], dtype=numpy.float64, copy=True)
    nq = numpy.dot(q, q)
    if nq < _EPS:
        return numpy.identity(4)
    q *= math.sqrt(2.0 / nq)
    q = numpy.outer(q, q)
    return numpy.array((
        (1.0-q[1, 1]-q[2, 2],     q[0, 1]-q[2, 3],     q[0, 2]+q[1, 3], 0.0),
        (    q[0, 1]+q[2, 3], 1.0-q[0, 0]-q[2, 2],     q[1, 2]-q[0, 3], 0.0),
        (    q[0, 2]-q[1, 3],     q[1, 2]+q[0, 3], 1.0-q[0, 0]-q[1, 1], 0.0),
        (                0.0,                 0.0,                 0.0, 1.0)
        ), dtype=numpy.float64)

def euler_from_matrix(matrix, axes='sxzy'):
    """Return Euler angles from rotation matrix for specified axis sequence.

    axes : One of 24 axis sequences as string or encoded tuple

    Note that many Euler angle triplets can describe one matrix.

    >>> R0 = euler_matrix(1, 2, 3, 'syxz')
    >>> al, be, ga = euler_from_matrix(R0, 'syxz')
    >>> R1 = euler_matrix(al, be, ga, 'syxz')
    >>> numpy.allclose(R0, R1)
    True
    >>> angles = (4.0*math.pi) * (numpy.random.random(3) - 0.5)
    >>> for axes in _AXES2TUPLE.keys():
    ...    R0 = euler_matrix(axes=axes, *angles)
    ...    R1 = euler_matrix(axes=axes, *euler_from_matrix(R0, axes))
    ...    if not numpy.allclose(R0, R1): print axes, "failed"

    """
    try:
        firstaxis, parity, repetition, frame = _AXES2TUPLE[axes.lower()]
    except (AttributeError, KeyError):
        _ = _TUPLE2AXES[axes]
        firstaxis, parity, repetition, frame = axes

    i = firstaxis
    j = _NEXT_AXIS[i+parity]
    k = _NEXT_AXIS[i-parity+1]

    M = numpy.array(matrix, dtype=numpy.float64, copy=False)[:3, :3]
    if repetition:
        sy = math.sqrt(M[i, j]*M[i, j] + M[i, k]*M[i, k])
        if sy > _EPS:
            ax = math.atan2( M[i, j],  M[i, k])
            ay = math.atan2( sy,       M[i, i])
            az = math.atan2( M[j, i], -M[k, i])
        else:
            ax = math.atan2(-M[j, k],  M[j, j])
            ay = math.atan2( sy,       M[i, i])
            az = 0.0
    else:
        cy = math.sqrt(M[i, i]*M[i, i] + M[j, i]*M[j, i])
        if cy > _EPS:
            ax = math.atan2( M[k, j],  M[k, k])
            ay = math.atan2(-M[k, i],  cy)
            az = math.atan2( M[j, i],  M[i, i])
        else:
            ax = math.atan2(-M[j, k],  M[j, j])
            ay = math.atan2(-M[k, i],  cy)
            az = 0.0

    if parity:
        ax, ay, az = -ax, -ay, -az
    if frame:
        ax, az = az, ax
    return ax, ay, az


def euler_from_quaternion(quaternion, axes='sxyz'):
    """Return Euler angles from quaternion for specified axis sequence.

    >>> angles = euler_from_quaternion([0.06146124, 0, 0, 0.99810947])
    >>> numpy.allclose(angles, [0.123, 0, 0])
    True

    """
    return euler_from_matrix(quaternion_matrix(quaternion), axes)

In [2]:
from Arduino import Arduino
import time
from rift import PyRift
import numpy as np

board = Arduino('9600') #plugged in via USB, serial com at rate 9600
board.pinMode(13, "OUTPUT")
board.Servos.attach(6) #declare servo on pin 6
board.Servos.attach(7) #declare servo on pin 7
board.Servos.attach(5) #declare servo on pin 5
n=0
foo = PyRift()
while(True):
    n+=1
    
    foo.poll()
    
    angles = euler_from_quaternion([foo.rotation[0], foo.rotation[1], foo.rotation[2], foo.rotation[3]])
    angle1 = int(angles[0]*180.0/np.pi)+100
    angle2 = -int(angles[1]*180.0/np.pi)+50
    angle3 = -int(angles[2]*180.0/np.pi)+50
    
    print angle1, angle2, angle3
    
    board.Servos.write(6, angle1) #move servo on pin 9 to 0 degrees
    time.sleep(0.001)
    board.Servos.write(7, angle2) #move servo on pin 9 to 0 degrees
    time.sleep(0.001)
    board.Servos.write(5, angle3) #move servo on pin 9 to 0 degrees
    
board.Servos.detach(6)
board.Servos.detach(7)
board.Servos.detach(5)

100 50 50
100 50 50
100 50 50
100 50 50
100 50 50
100 50 50
100 50 50
100 50 50
101 50 50
101 50 50
101 50 50
101 50 50
101 50 50
101 50 50
101 50 50
101 50 50
102 50 50
102 50 50
102 50 50
102 50 51
102 50 51
102 50 52
101 49 52
100 48 53
99 48 53
99 47 53
97 44 54
96 43 54
96 42 54
96 42 54
96 41 53
96 41 53
96 42 53
96 42 53
98 45 53
98 45 53
98 46 54
98 48 54
98 48 54
98 49 54
97 50 54
97 50 54
97 50 53
95 50 52
95 50 52
95 50 52
94 50 51
94 50 52
95 51 52
98 51 55
98 51 56
99 50 56
96 52 61
95 53 61
94 53 61
94 54 61
93 54 61
92 54 61
91 54 61
90 53 61
89 50 59
88 50 58
88 50 58
92 50 61
93 50 61
93 50 61
97 52 60
97 52 59
96 52 59
96 52 58
96 52 58
96 52 58
96 52 58
97 52 58
97 51 58
97 51 58
97 51 58
97 51 58
97 51 58
97 51 58
97 51 57
97 51 57
97 51 57
97 52 57
97 52 57
97 52 57
97 52 57
97 52 57
97 52 57
97 52 57
97 52 57
97 52 57
97 52 58
97 52 57
97 52 58
97 52 58
97 52 58
97 52 58
97 52 58
97 52 58
97 52 58
97 52 58
97 52 58
97 52 58
97 52 58
97 52 58
97 52 58
97 52 58
97 5

KeyboardInterrupt: 